In [2]:
import torch
import time

In [66]:

@torch.no_grad()
def seed_gen_rad_mat(rm_size, feat_size, device, dtype, seed):
    torch.cuda.manual_seed(seed)
    bern = torch.randint(2, size=rm_size, device=device, requires_grad=False, dtype=dtype)
    return (2.0 * bern - 1) / feat_size **0.5
@torch.no_grad()
def low_mem_input2rp(input, kept_acts):
    assert len(input.size()) == 2
    rand_mat_size = (input.shape[1], kept_acts)
    # Create random matrix
    seed = int(time.time()*1000)
    rand_matrix = seed_gen_rad_mat(rand_mat_size, kept_acts, input.device, input.dtype, seed)
    dim_reduced_input = torch.matmul(input, rand_matrix)
    print('='*20, 'input2rp', '='*20)
    print(rand_matrix)
    return dim_reduced_input, rand_mat_size, seed


@torch.no_grad()
def low_mem_rp2input(dim_reduced_input, input_shape, seed, rm_size):
    assert len(dim_reduced_input.size()) == 2
    rand_matrix = seed_gen_rad_mat(rm_size, rm_size[1], dim_reduced_input.device, dim_reduced_input.dtype, seed)
    input = torch.matmul(dim_reduced_input, rand_matrix.t())    
    print('='*20, 'rp2input', '='*20)
    print(rand_matrix)
    return input.view(input_shape)

In [72]:
input = torch.rand((4,4), dtype=torch.float32, device='cuda')
kept_acts = 2
dim_reduced_input, rand_mat_size, seed = low_mem_input2rp(input, kept_acts)
input_recovered = low_mem_rp2input(dim_reduced_input, input.shape, seed, rand_mat_size)


==================== input2rp ====================
tensor([[ 0.7071, -0.7071],
        [-0.7071,  0.7071],
        [ 0.7071,  0.7071],
        [ 0.7071, -0.7071]], device='cuda:0')
==================== rp2input ====================
tensor([[ 0.7071, -0.7071],
        [-0.7071,  0.7071],
        [ 0.7071,  0.7071],
        [ 0.7071, -0.7071]], device='cuda:0')


In [49]:
seed = int(time.time()*1000)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)


In [64]:
# torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
print(torch.rand(4,4, device='cuda'))

tensor([[0.6944, 0.0137, 0.4753, 0.3608],
        [0.7167, 0.2386, 0.7950, 0.8708],
        [0.8432, 0.6082, 0.2663, 0.2225],
        [0.3606, 0.7359, 0.5603, 0.2655]], device='cuda:0')
